In [ ]:
# 운영체제(OS)와 상호작용하기 위한 기본 파이썬 라이브러리
# 환경 변수 읽기, 경로 처리 등에 사용됨
import os

# .env 파일을 읽어서 환경 변수로 등록해주는 라이브러리에서
# load_dotenv 함수만 가져옴
from dotenv import load_dotenv

# 현재 프로젝트 폴더에 있는 .env 파일을 찾아서
# 내부의 KEY=VALUE 형태의 값을
# 운영체제 환경 변수로 등록함
load_dotenv()

# 운영체제 환경 변수 중에서
# "OPENAI_API_KEY" 라는 이름의 값을 가져옴
# 만약 없다면 None 이 반환됨
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [2]:
pip install pydantic

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langchain openai langchain-openai langchain-community

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 27.0 MB/s eta 0:00:00

   ---------- ----------------------------- 2/8 [requests]
   ---------- ----------------------------- 2/8 [requests]
   ---------- ----------------------------- 2/8 [requests]
   ---------- ----------------------------- 2/8 [requests]
   --------------- ------------------------ 3/8 [marshmallow]
   --------------- ------------------------ 3/8 [marshmallow]
   -------------------- ------------------- 4/8 [httpx-sse]
   ------------------------- -------------- 5/8 [dataclasses-json]
   ------------------------------ --------- 6/8 [pydantic-settings]
   ------------------------------ --------- 6/8 [pydantic-settings]
   ------------------------------ --------- 6/8 [pydantic-settings]
   ------------------------------ --------- 6/8 [pydantic-settings]
   -------------

In [ ]:
# LangChain의 document_loaders 모듈에서
# GitHub 저장소를 불러오기 위한 GitLoader 클래스를 가져온다
# (GitHub 저장소를 clone해서 문서 형태로 읽어오는 역할)
from langchain_classic.document_loaders import GitLoader

# 어떤 파일을 불러올지 결정하는 "필터 함수"를 직접 만든다
# GitLoader는 저장소의 모든 파일 경로를 하나씩 이 함수에 넣어 검사한다
def file_filter(file_path):
    # file_path: 파일의 전체 경로 문자열
    # endswith(".md")는 해당 파일명이 ".md"로 끝나는지 검사한다
    # 예: README.md -> True
    # 예: main.py   -> False

    # True 를 반환하면 -> 이 파일을 읽음
    # False 를 반환하면 -> 이 파일을 무시함
    return file_path.endswith(".md")

# GitLoader 객체 생성 (아직 실행 아님, 설정만 하는 단계)
loader=GitLoader(
     # clone_url:
    # GitHub에서 가져올 저장소 주소
    # 내부적으로 git clone 명령을 자동 실행함
    clone_url="https://github.com/langchain-ai/langchain",

    # repo_path:
    # 저장소를 다운로드해서 저장할 로컬 폴더 위치
    # "./langchain" 은 현재 파이썬 파일 위치에
    # langchain 이라는 폴더를 생성해서 저장한다는 의미
    repo_path="./langchain",

     # branch:
    # GitHub 저장소의 사용할 브랜치 이름
    # master 브랜치 기준으로 파일을 가져옴
    branch="master",

     # file_filter:
    # 위에서 만든 file_filter 함수를 연결
    # GitLoader가 모든 파일을 검사할 때 사용됨
    file_filter=file_filter,
)

# 실제로 GitHub 저장소를 clone 하고
# 파일을 읽어서
# LangChain Document 객체 리스트로 변환하는 실행 명령
raw_docs=loader.load()
# 불러온 문서(Document) 개수를 출력
# raw_docs는 리스트(list)이므로 len()으로 개수 확인 가능
print(len(raw_docs))

36


## Document transformers

In [ ]:
# LangChain에서 제공하는 텍스트 분할 도구(Text Splitter) 중
# "글자 수(Character) 기준"으로 문서를 자르는
# CharacterTextSplitter 클래스를 가져온다
from langchain_classic.text_splitter import CharacterTextSplitter

# CharacterTextSplitter 객체 생성 (설정 단계)
# 아직 문서를 자르지는 않고 "어떻게 자를지 규칙"만 정의한다

                                # chunk_size=1000
                                # 하나의 문서 조각(chunk)의 최대 길이를 1000 "글자(character)"로 설정
                                # 즉, 문서가 길면 1000글자 단위로 잘라서 여러 조각으로 나눔
text_splitter=CharacterTextSplitter(chunk_size=1000,
                                    # chunk_overlap=0
                                    # 이전 조각과 다음 조각이 서로 겹치는 글자 수
                                    # 0이면 겹치지 않고 딱 나누어서 분할함
                                    # (예: 1~1000, 1001~2000 ...)
                                    chunk_overlap=0)

# raw_docs에 들어있는 Document 객체 리스트를 입력으로 받아서
# 설정한 규칙(1000글자 단위)대로 잘게 나눈다
# 결과는 "분할된 Document 리스트" 형태로 반환된다
docs=text_splitter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 1367, which is longer than the specified 1000
Created a chunk of size 1455, which is longer than the specified 1000
Created a chunk of size 1299, which is longer than the specified 1000


96


In [ ]:
# LangChain에서 OpenAI 임베딩(Embedding) 기능을 사용하기 위한 클래스 import
# 텍스트를 숫자 벡터(의미 벡터)로 변환할 때 사용됨
from langchain_openai import OpenAIEmbeddings


# OpenAIEmbeddings 객체 생성
# 내부적으로:
# 1. 환경변수에서 OPENAI_API_KEY 읽음
# 2. OpenAI Embedding 모델 설정
# 3. OpenAI 서버와 통신할 준비를 함
# (아직 API 호출은 발생하지 않음)
embeddings=OpenAIEmbeddings()



# --------------------------------------------
# 아래 명령은 "터미널"에서 실행해야 함
# --------------------------------------------
# OpenAI 토큰 계산 라이브러리 설치
# LangChain이 입력 텍스트의 토큰 수 계산 및 길이 관리에 사용함
# (없으면 embedding 호출 시 오류 발생 가능)
#
# 터미널에서 실행:
# pip install tiktoken
# --------------------------------------------

In [5]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 임베딩(벡터화) 할 텍스트 질문
# 보통 실제 서비스에서는 사용자가 입력한 질문이 들어감
query="AWS의 S3에서 데이터를 읽기 위한 DocumentLoader가 있나요?"


# query 문자열을 OpenAI Embedding API로 보내서
# 의미를 숫자 벡터 형태로 변환함
#
# 내부 동작:
# 1. query 텍스트를 OpenAI 서버로 전송
# 2. Embedding 모델 실행
# 3. 수천 차원의 실수(float) 벡터 반환
vector=embeddings.embed_query(query)


# 생성된 벡터의 길이(차원 수) 출력
# 보통 1536 또는 3072 차원이 나옴
print(len(vector))


# 벡터 전체는 너무 길기 때문에
# 앞에서부터 100개 숫자만 출력해서 확인
print(vector[:100])

1536
[-0.01556143257766962, -0.012867593206465244, 0.021737877279520035, -0.01614966429769993, -0.024117546156048775, 0.045186977833509445, -0.019705800339579582, -0.0077606625854969025, -0.0040641543455421925, 0.022138945758342743, 0.01052134670317173, 0.01549458783119917, 0.0010402697371318936, -0.018796712160110474, -0.007393017411231995, -0.002800790360197425, 0.0177271980792284, -0.00953204557299614, -0.009244613349437714, -0.03665760159492493, 0.012967860326170921, -0.015628276392817497, -0.005621634423732758, 0.0017429740400984883, -0.0026554034557193518, 0.009017341770231724, 0.007687133736908436, -0.04016026109457016, 0.007219220977276564, 0.0026470478624105453, 0.0526735782623291, -0.013041389174759388, -0.0068415491841733456, -0.022727178409695625, -0.010026696138083935, -0.014705820940434933, 0.00838231761008501, -0.0006972731789574027, -0.006173102650791407, -0.02152397483587265, 0.002012023702263832, -0.006493956781923771, -0.0006116284639574587, -0.04347575455904007, 0.0

In [ ]:
# --------------------------------------------
# 터미널에서 먼저 실행해야 함 (1회만)
# --------------------------------------------
# Chroma 벡터 데이터베이스 설치
# 임베딩 벡터를 저장하고
# 의미 기반 검색(similarity search)을 하기 위한 DB
#
# 터미널 실행:
# pip install chromadb
# --------------------------------------------
pip install chromadb

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/21.4 MB ? eta -:--:--
   ------- -------------------------------- 3.9/21.4 MB 19.8 MB/s eta 0:00:01
   -------------- ------------------------- 7.6/21.4 MB 19.3 MB/s eta 0:00:01
   --------------------------- ------------ 14.9/21.4 MB 24.8 MB/s eta 0:00:01
   ---------------------------------------  21.0/21.4 MB 25.8 MB/s eta 0:00:01
   ---------------------------------------- 21.4/21.4 MB 24.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 37.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
   ----------------- ---------------------- 6.0/13.5 MB 30.5 MB/s eta 0:00:01
   ----------------------------------- ---- 11.8/13.5 MB 31.6 MB/s eta 0:00:01
   ---------------------------------------- 13.5/13.5 MB 23.2 MB/s eta 0:00:00
   ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [ ]:
# LangChain에서 Chroma 벡터스토어 클래스를 import
# 벡터를 저장하고 검색하는 역할
from langchain_classic.vectorstores import Chroma


# docs : 이전 단계에서 chunk 분할된 Document 리스트
# embeddings : OpenAIEmbeddings 객체
#
# from_documents() 는:
# 1. docs 텍스트를 임베딩으로 변환
# 2. Chroma DB에 벡터 저장
# 3. 검색 가능한 DB 객체 생성
db=Chroma.from_documents(docs,embeddings)

## Retriever

In [ ]:
# Vector DB를 "검색기(Retriever)" 형태로 변환
# Retriever는 자연어 질문을 받아서
# 관련 문서를 자동으로 찾아주는 인터페이스
retriever=db.as_retriever()

In [ ]:
# 사용자가 입력한 질문 (검색용 질의)
query="AWS S3에서 데이터를 불러올수있는 DocumentLoader가 있나요?"

# retriever.invoke(query)
# → query를 임베딩으로 변환
# → DB에 저장된 벡터들과 거리 비교
# → 가장 비슷한 문서들을 찾아서 반환
context_docs=retriever.invoke(query)

# 검색 결과로 반환된 문서 개수 출력
print(f"eln ={len(context_docs)}")

# 검색된 문서 중 가장 첫 번째 문서 선택
# (가장 유사도가 높은 문서)
first_doc=context_docs[0]

# 해당 문서의 메타데이터 출력
# (원본 파일 경로, 저장소 정보 등)
print(f"metadata ={ first_doc.metadata}")

# 해당 문서의 실제 내용(텍스트) 출력
print(first_doc.page_content)

eln =4
metadata ={'file_type': '.md', 'file_name': 'README.md', 'source': '.devcontainer\\README.md', 'file_path': '.devcontainer\\README.md'}
Then you will have a local cloned repo where you can contribute and then create pull requests.

If you already have VS Code and Docker installed, you can use the button above to get started. This will use VSCode to automatically install the Dev Containers extension if needed, clone the source code into a container volume, and spin up a dev container for use.

Alternatively you can also follow these steps to open this repo in a container using the VS Code Dev Containers extension:

1. If this is your first time using a development container, please ensure your system meets the pre-reqs (i.e. have Docker installed) in the [getting started steps](https://aka.ms/vscode-remote/containers/getting-started).

2. Open a locally cloned copy of the code:


## RetrievalQA(Chain)

In [ ]:
# LangChain에서 RetrievalQA 체인 클래스를 import
# Retrieval(문서 검색) + QA(질의응답)를 하나로 묶은 파이프라인
from langchain_classic.chains import RetrievalQA
# OpenAI Chat 모델(GPT)을 LangChain에서 사용하기 위한 클래스 import
from langchain_openai import ChatOpenAI

# ChatGPT 모델 객체 생성
# model_name: 사용할 OpenAI 모델 이름
# temperature=0 : 출력의 랜덤성 제거 (항상 최대한 동일한 답변 생성)
llm=ChatOpenAI(model_name="gpt-4.1-mini",temperature=0)

# RetrievalQA 체인 생성
# llm: 답변 생성 담당 (GPT)
# retriever: 벡터 DB에서 관련 문서 검색 담당
# chain_type='stuff': 검색된 문서를 전부 하나로 합쳐서 프롬프트에 넣는 방식
qa_chain=RetrievalQA.from_chain_type(llm=llm,  # GPT 모델 객체
                                     chain_type='stuff',
                                    # 체인 타입
                                    # 'stuff' = 검색된 문서들을 그대로 다 합쳐서 프롬프트에 "통째로 넣기"
                                     retriever=retriever)
                                         # 앞에서 만든 retriever (Chroma 기반)
# 질문 실행
# 내부에서:
# 1. 질문 임베딩 생성
# 2. 벡터 DB 검색
# 3. 관련 문서 수집
# 4. 문서 + 질문을 GPT 프롬프트로 구성
# 5. GPT 답변 생성
qa_chain.invoke(query)

{'query': 'AWS S3에서 데이터를 불러올수있는 DocumentLoader가 있나요?',
 'result': '네, Langchain에는 AWS S3에서 데이터를 불러올 수 있는 DocumentLoader가 있습니다.\n\nLangchain의 `langchain.document_loaders` 모듈 내에 `S3FileLoader`라는 클래스가 있어, AWS S3 버킷에 저장된 파일을 로드할 수 있습니다.\n\n사용 예시는 다음과 같습니다:\n\n```python\nfrom langchain.document_loaders import S3FileLoader\n\n# S3 버킷 내 파일 경로 지정 (예: \'s3://bucket-name/path/to/file.txt\')\ns3_path = "s3://your-bucket-name/path/to/your-file.txt"\n\nloader = S3FileLoader(s3_path)\ndocuments = loader.load()\n```\n\n이때 AWS 자격증명(AWS Access Key, Secret Key 등)은 환경 변수나 AWS CLI 설정을 통해 미리 설정되어 있어야 합니다.\n\n더 자세한 내용과 지원되는 파일 형식은 Langchain 공식 문서의 [통합 문서](https://docs.langchain.com/oss/python/integrations/providers)에서 확인할 수 있습니다.'}